# Abstract 



The purpose of this project is to find the important hyperparameters for different models.Hyperparameters are the parameters that are specified externally for tuning a predictive modeling problem. They tell the model how fast it should run, how much depth the tree should go etc. We cannot detect the best value of hyperparameter for the model, so we do trial and error method by randomly choosing and running the values. This is a hectic task for data scientists as it is time consuming process. Hence,a database should be  created containing the information of best hyperparameters for all the algorithms for different runtimes. So in this project, we use H2O AutoML to generate best models for six different runtimes - 333,500,700,1300,1500 and the parameters for each model is stored in JSON file and sent to database team to store those parameters in a database. And after getting the csv file from database from which has all the hyperparameter values for all the models with all runtimes together, analysis has been done to find important hyperparameters and to find the range of each hyperparameter in all the models and also compared how the hyperparameter values differ in each algorithm    

### Data Exploration

Importing the H20, pandas, csv, json package which is used for data manupulation and for storing model in csv and json. H2O is an open source software in machine learning platflorm 

In [23]:
#importing t
import h2o
from h2o.automl import H2OAutoML
import random, os, sys
from datetime import datetime
import pandas as pd
import logging
import csv
import optparse
import time
import json
from distutils.util import strtobool
import psutil

import warnings
warnings.filterwarnings('ignore')



In [24]:
port_no=random.randint(5555,55555)
h2o.init(strict_version_check=False,min_mem_size_GB=5,port=port_no)

Checking whether there is an H2O instance running at http://localhost:19029..... not found.
Attempting to start a local H2O server...
  Java Version: java version "1.8.0_101"; Java(TM) SE Runtime Environment (build 1.8.0_101-b13); Java HotSpot(TM) 64-Bit Server VM (build 25.101-b13, mixed mode)
  Starting server from /Applications/anaconda3/lib/python3.7/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /var/folders/gf/p944hdxd7jq73_9cxfglxn540000gn/T/tmpshy20khm
  JVM stdout: /var/folders/gf/p944hdxd7jq73_9cxfglxn540000gn/T/tmpshy20khm/h2o_vaish_started_from_python.out
  JVM stderr: /var/folders/gf/p944hdxd7jq73_9cxfglxn540000gn/T/tmpshy20khm/h2o_vaish_started_from_python.err
  Server is running at http://127.0.0.1:19029
Connecting to H2O server at http://127.0.0.1:19029... successful.


H2O cluster uptime:,02 secs
H2O cluster timezone:,America/New_York
H2O data parsing timezone:,UTC
H2O cluster version:,3.22.1.6
H2O cluster version age:,1 month and 12 days
H2O cluster name:,H2O_from_python_vaish_zp0pin
H2O cluster total nodes:,1
H2O cluster free memory:,4.792 Gb
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster status:,"accepting new members, healthy"


##### For the H2o server to tun, a port number  is chosen between the random numbers from 5555 and 55555 and initialized. Memory size of 5 GB is given for h2o.

In [25]:
#importing the file
df = h2o.import_file(path="anime_filtered.csv")

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [26]:
###### the given dataset is loaded into the dataframe called df

In [27]:
# Functions

def alphabet(n):
    alpha='0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ'    
    str=''
    r=len(alpha)-1   
    while len(str)<n:
        i=random.randint(0,r)
        str+=alpha[i]   
    return str
  
def set_meta_data(run_id,data,target,run_time,regression,name,path,min_mem_size):
      m_data={}
      m_data['start_time'] = time.time()
      m_data['target']=target
      m_data['data_path']=data 
      m_data['run_time']=run_time
      m_data['run_id'] =run_id
      m_data['regression']=regression
      m_data['scale']=False
      m_data['project'] =name
      m_data['end_time'] = time.time()
      m_data['execution_time'] = 0.0
      m_data['run_path'] =path
      m_data['min_mem_size'] = min_mem_size
      m_data['balance_threshold'] = 0.2
      return m_data


def impute_missing_values(df, x, scal=False):
    # determine column types
    ints, reals, enums = [], [], []
    for key, val in df.types.items():
        if key in x:
            if val == 'enum':
                enums.append(key)
            elif val == 'int':
                ints.append(key)            
            else: 
                reals.append(key)    
    _ = df[reals].impute(method='mean')
    _ = df[ints].impute(method='median')
    if scal:
        df[reals] = df[reals].scale()
        df[ints] = df[ints].scale()    
    return


def get_independent_variables(df, targ):
    C = [name for name in df.columns if name != targ]
    # determine column types
    ints, reals, enums = [], [], []
    for key, val in df.types.items():
        if key in C:
            if val == 'enum':
                enums.append(key)
            elif val == 'int':
                ints.append(key)            
            else: 
                reals.append(key)    
    x=ints+enums+reals
    return x
    
def get_all_variables_csv(i):
    ivd={}
    try:
      iv = pd.read_csv(i,header=None)
    except:
      sys.exit(1)    
    col=iv.values.tolist()[0]
    dt=iv.values.tolist()[1]
    i=0
    for c in col:
      ivd[c.strip()]=dt[i].strip()
      i+=1        
    return ivd
    
    

def check_all_variables(df,dct,y=None):     
    targ=list(dct.keys())     
    for key, val in df.types.items():
        if key in targ:
          if dct[key] not in ['real','int','enum']:                      
            targ.remove(key)  
    for key, val in df.types.items():
        if key in targ:            
          if dct[key] != val:
            print('convert ',key,' ',dct[key],' ',val)
            if dct[key]=='enum':
                try:
                  df[key] = df[key].asfactor() 
                except:
                  targ.remove(key)                 
            if dct[key]=='int': 
                try:                
                  df[key] = df[key].asnumeric() 
                except:
                  targ.remove(key)                  
            if dct[key]=='real':
                try:                
                  df[key] = df[key].asnumeric()  
                except:
                  targ.remove(key)                  
    if y is None:
      y=df.columns[-1] 
    if y in targ:
      targ.remove(y)
    else:
      y=targ.pop()            
    return targ    


def check_X(x,df):
    for name in x:
        if name not in df.columns:
          x.remove(name)  
    return x    
    
    

def get_variables_types(df):
    d={}
    for key, val in df.types.items():
        d[key]=val           
    return d    
    
#  End Functions

#### Functions defined for the data

- alphabet(n): function generates a random string ranging from a to z, 0to 9, A to Z.
- set_meta_data : function provide the basic meta data properties of the variable.It contains the details of run_id,model start time, model execution time
- impute_missing_values : function is used to imute the missing values by placing them with the mean and median values corresponding to their data types.
- get_independent_variables : function generate all the independent variables, ignoring the target value 
- get_all_variables_csv : function provides all the variable values
- check_all_variables : function checks on the basis of datatypes, independent variables and target values 
- check_X : function checks the value of x, if it is not present in the guven dataset, value is removed
- get_variables_types : function tell us the data type of the different variables

    


__Displaying the top 5 rows of the dataset__

In [28]:
df.head()



anime_id,title,title_english,title_japanese,title_synonyms,image_url,type,source,episodes,status,airing,aired_string,aired,duration,rating,score,scored_by,rank,popularity,members,favorites,background,premiered,broadcast,related,producer,licensor,studio,genre,opening_theme,ending_theme
11013,Inu x Boku SS,Inu X Boku Secret Service,妖狐×僕SS,Youko x Boku SS,https://myanimelist.cdn-dena.com/images/anime/12/35893.jpg,TV,Manga,12,Finished Airing,False,"Jan 13, 2012 to Mar 30, 2012","{'from': '2012-01-13', 'to': '2012-03-30'}",24 min. per ep.,PG-13 - Teens 13 or older,7.63,139250,1274,231,283882,2809,"Inu x Boku SS was licensed by Sentai Filmworks for North America, while MVM Films licensed it for the United Kingdom. During April 2013, Hanabee Entertainment released the series on DVD and Blu-ray for Australia and New Zealand.",Winter 2012,Fridays at Unknown,"{'Adaptation': [{'mal_id': 17207, 'type': 'manga', 'url': 'https://myanimelist.net/manga/17207/Inu_x_Boku_SS', 'title': 'Inu x Boku SS'}], 'Sequel': [{'mal_id': 13403, 'type': 'anime', 'url': 'https://myanimelist.net/anime/13403/Inu_x_Boku_SS_Special', 'title': 'Inu x Boku SS Special'}]}","Aniplex, Square Enix, Mainichi Broadcasting System, Movic, Inu x Boku SS Production Partners",Sentai Filmworks,David Production,"Comedy, Supernatural, Romance, Shounen","['""Nirvana"" by MUCC']","['#1: ""Nirvana"" by MUCC (eps 1, 11-12)', '#2: ""Rakuen no Photograph (楽園のPhotograph)"" by Soushi Miketsukami (Yuichi Nakamura) (eps 2, 9)', '#3: ""Kimi wa (君は)"" by Ririchiyo Shirakiin (Rina Hidaka) (eps 3, 8)', '#4: ""one way"" by Banri Watanuki (Takuya Eguchi) & Zange Natsume (Mamoru Miyano) (ep 4)', '#5: ""SM Hantei Forum (SM判定フォーラム)"" by Kagerou Shoukiin (Tomokazu Sugita) (ep 5, 10)', '#6: ""sweets parade"" by Karuta Roromiya (Kana Hanazawa) (ep 6)', '#7: ""Taiyou to Tsuki (太陽と月)"" by Renshou Sorinozuka (Yoshimasa Hosoya) & Nobara Yukinokouji (Youko Hikasa) (ep 7)']"
2104,Seto no Hanayome,My Bride is a Mermaid,瀬戸の花嫁,The Inland Sea Bride,https://myanimelist.cdn-dena.com/images/anime/13/58383.jpg,TV,Manga,26,Finished Airing,False,"Apr 2, 2007 to Oct 1, 2007","{'from': '2007-04-02', 'to': '2007-10-01'}",24 min. per ep.,PG-13 - Teens 13 or older,7.89,91206,727,366,204003,2579,,Spring 2007,Unknown,"{'Adaptation': [{'mal_id': 759, 'type': 'manga', 'url': 'https://myanimelist.net/manga/759/Seto_no_Hanayome', 'title': 'Seto no Hanayome'}], 'Sequel': [{'mal_id': 4163, 'type': 'anime', 'url': 'https://myanimelist.net/anime/4163/Seto_no_Hanayome_OVA', 'title': 'Seto no Hanayome OVA'}], 'Side story': [{'mal_id': 5199, 'type': 'anime', 'url': 'https://myanimelist.net/anime/5199/Seto_no_Hanayome_Fan_Disc', 'title': 'Seto no Hanayome Fan Disc'}], 'Alternative version': [{'mal_id': 28385, 'type': 'anime', 'url': 'https://myanimelist.net/anime/28385/Seto_no_Hanayome_Picture_Drama', 'title': 'Seto no Hanayome Picture Drama'}]}","TV Tokyo, AIC, Square Enix, Sotsu",Funimation,Gonzo,"Comedy, Parody, Romance, School, Shounen","['""Romantic summer"" by SUN&LUNAR']","['#1: ""Ashita e no Hikari (明日への光)"" by Asuka Hinoi (1-13, 26)', '#2: ""Dan Dan Dan"" by SUN&LUNAR (eps 14-25)']"
5262,Shugo Chara!! Doki,Shugo Chara!! Doki,しゅごキャラ！！どきっ,"Shugo Chara Ninenme, Shugo Chara! Second Year",https://myanimelist.cdn-dena.com/images/anime/11/10645.jpg,TV,Manga,51,Finished Airing,False,"Oct 4, 2008 to Sep 25, 2009","{'from': '2008-10-04', 'to': '2009-09-25'}",24 min. per ep.,PG - Children,7.55,37129,1508,1173,70127,802,,Fall 2008,Unknown,"{'Adaptation': [{'mal_id': 101, 'type': 'manga', 'url': 'https://myanimelist.net/manga/101/Shugo_Chara', 'title': 'Shugo Chara!'}], 'Prequel': [{'mal_id': 2923, 'type': 'anime', 'url': 'https://myanimelist.net/anime/2923/Shugo_Chara', 'title': 'Shugo Chara!'}], 'Sequel': [{'mal_id': 7082, 'type': 'anime', 'url': 'https://myanimelist.net/anime/7082/Shugo_Chara_Party', 'title': 'Shugo Chara! Party!'}]}","TV Tokyo, Sotsu",,Satelight,"Comedy, Magic, School, Shoujo","['#1: ""Minna no Tamago (みんなのたまご)"" by S

Some variables has been defined which is used to store in meta data 

In [29]:
all_variables=None
test_path=None
nthreads=1 
regression=True   
model_path=None
balance_y=False 
balance_threshold=0.2
name=None 
server_path=None  
target = 'score'
min_mem_size = 5;
run_time=0
data_path="anime_filtered.csv"
project ="automl_test"

In [30]:
#finding the datatypes of Nan values
df.types

{'anime_id': 'int',
 'title': 'string',
 'title_english': 'string',
 'title_japanese': 'string',
 'title_synonyms': 'string',
 'image_url': 'string',
 'type': 'enum',
 'source': 'enum',
 'episodes': 'int',
 'status': 'enum',
 'airing': 'enum',
 'aired_string': 'enum',
 'aired': 'enum',
 'duration': 'enum',
 'rating': 'enum',
 'score': 'real',
 'scored_by': 'int',
 'rank': 'int',
 'popularity': 'int',
 'members': 'int',
 'favorites': 'int',
 'background': 'string',
 'premiered': 'enum',
 'broadcast': 'enum',
 'related': 'enum',
 'producer': 'enum',
 'licensor': 'enum',
 'studio': 'enum',
 'genre': 'enum',
 'opening_theme': 'enum',
 'ending_theme': 'enum'}

__Data Cleaning__

Data cleaning is done in the model by removing all Nan values.

In [31]:
#calculating the sum of Nan values
df.isna().sum()

73271.0

__Finding number of missing values__

In [32]:
episodes = df['episodes'].isna().sum()
score = df['score'].isna().sum()
rank = df['rank'].isna().sum()
popularity = df['popularity'].isna().sum()
memebers = df['members'].isna().sum()
favorites = df['favorites'].isna().sum()

print("Number of missing values in each numeric column:")
print("Episodes : "+str(episodes))
print("Score : "+str(score))
print("Rank  : "+str(rank))
print("Popularity :"+str(popularity))
print("Members :"+str(memebers))
print("Favorites :"+str(favorites))
    

Number of missing values in each numeric column:
Episodes : 0.0
Score : 0.0
Rank  : 1573.0
Popularity :0.0
Members :0.0
Favorites :0.0


__Fill the missing values with mean value__

In [33]:
df['rank'] = df['rank'].impute(method='mean')

__Independent variable function is called and X is initialized with all the variables except target and y is initialized with target__

In [34]:


X = []  
if all_variables is None:
  X=get_independent_variables(df, target)  
else: 
  ivd=get_all_variables_csv(all_variables)    
  X=check_all_variables(df, ivd)


X=check_X(X,df)



In [35]:
y=target

__An automl function is called which takes x,y,data variables and run it using AutoML that automatically runs through all the algorithms and their hyperparameters to produce a leaderboard of the best models__

In [36]:
def automl(maxruntime,X,Y,df):
    aml = H2OAutoML(max_runtime_secs=maxruntime)
    aml.train(x=X,y=y,training_frame=df)
    return aml

__dict_to_json function is defined to generate json files with hyperparameters of every model generated for each runtime__

In [37]:
def dict_to_json(dct,n):  
    j = json.dumps(dct, indent=4)
    f = open(n, 'w')
    print(j, file=f)
    f.close()

## Run Time = 333 seconds

In [38]:
run_id=alphabet(9)
if server_path==None:
    server_path=os.path.abspath(os.curdir)
os.chdir(server_path) 
run_dir = os.path.join(server_path,run_id)
os.mkdir(run_dir)
os.chdir(run_dir)    

name = run_id+'Anime'+str('333')
print (run_id)


fGcx8PAiA


#### Analysis from the above code

- Random run_id is generated randomly with the help of alphabet function defined above. 
- Server_path variable is initialized with present directory in which the server is running. 
- A folder is created with the name of run_id to store all the files generated with this run time.

In [39]:
meta_data = set_meta_data(run_id,data_path,target,run_time,regression,name,run_dir,min_mem_size)
print(meta_data)

{'start_time': 1556319415.292062, 'target': 'score', 'data_path': 'anime_filtered.csv', 'run_time': 0, 'run_id': 'fGcx8PAiA', 'regression': True, 'scale': False, 'project': 'fGcx8PAiAAnime333', 'end_time': 1556319415.292063, 'execution_time': 0.0, 'run_path': '/Users/vaish/fGcx8PAiA', 'min_mem_size': 5, 'balance_threshold': 0.2}


__A meta data variable is created to store basic functionalities of the AutomML model. This contains the details of run_id,model start time, model execution time etc.__

In [40]:

meta_data['X']=X  
meta_data['start_time'] = time.time()
allV=get_variables_types(df)
meta_data['variables_datatypes']=allV

__All the independent variables of X and their datatypes are stored in meta_data__

In [41]:
aml = automl(333,X,y,df)

AutoML progress: |████████████████████████████████████████████████████████| 100%


__automl function is called by giving data,runtimeof 333 ,independent variables and target__ 

In [42]:
run_time = 333
meta_data['run_time'] = run_time
meta_data['end_time'] = time.time()
meta_data['execution_time'] = meta_data['end_time'] - meta_data['start_time']
  

__After the AutoML is executed, execution time of the model is calculated and stored in meta data__

In [43]:
aml_leaderboard_df=aml.leaderboard.as_data_frame()
aml_leaderboard_df

,model_id,auc,logloss,mean_per_class_error,rmse,mse


__These are models generated in an order from best to worst__


__Total number of models generated is also stored in meta data__

In [44]:
length = len(aml_leaderboard_df)
length
meta_data["models_generated"] = length

__Leaderboard is stored in csv files__

In [45]:
leaderboard_stats=run_id+'_leaderboard.csv'
aml_leaderboard_df.to_csv(leaderboard_stats)

__The leaderboard is converted to data frame object so that all model_ids are stored in a seperate list to get parameters__

In [46]:
aml_leaderboard_df=aml.leaderboard.as_data_frame()
model_set=aml_leaderboard_df['model_id']
model_set

Series([], Name: model_id, dtype: object)

__Total count of the models is obtained__

In [47]:
count = 0;
for i in model_set:
    count = count+1;

__Each model parameters are stored in json file along with meta_data__

In [48]:
for i in range(0,count):
    mod_best=h2o.get_model(model_set[i])
    parameters = mod_best.params
    n= str((model_set[i]))+'__333'
    dict_to_json(parameters,n)

In [49]:
# Update and save meta data
n=run_id+'_meta_data.json'
dict_to_json(meta_data,n)

In [50]:
meta_data

{'start_time': 1556319415.299701,
 'target': 'score',
 'data_path': 'anime_filtered.csv',
 'run_time': 333,
 'run_id': 'fGcx8PAiA',
 'regression': True,
 'scale': False,
 'project': 'fGcx8PAiAAnime333',
 'end_time': 1556319756.723257,
 'execution_time': 341.42355608940125,
 'run_path': '/Users/vaish/fGcx8PAiA',
 'min_mem_size': 5,
 'balance_threshold': 0.2,
 'X': ['anime_id',
  'episodes',
  'scored_by',
  'rank',
  'popularity',
  'members',
  'favorites',
  'type',
  'source',
  'status',
  'airing',
  'aired_string',
  'aired',
  'duration',
  'rating',
  'premiered',
  'broadcast',
  'related',
  'producer',
  'licensor',
  'studio',
  'genre',
  'opening_theme',
  'ending_theme',
  'title',
  'title_english',
  'title_japanese',
  'title_synonyms',
  'image_url',
  'background'],
 'variables_datatypes': {'anime_id': 'int',
  'title': 'string',
  'title_english': 'string',
  'title_japanese': 'string',
  'title_synonyms': 'string',
  'image_url': 'string',
  'type': 'enum',
  'sour

__Since all the files for the particular run id are generated, the directory is again changed to the server path__


In [51]:
os.chdir(server_path)

## Run Time = 500 seconds

In [52]:
run_id=alphabet(9)
if server_path==None:
    server_path=os.path.abspath(os.curdir)
os.chdir(server_path) 
run_dir = os.path.join(server_path,run_id)
os.mkdir(run_dir)
os.chdir(run_dir)    

name = run_id+'Anime'+str('500')
# run_id to std out
print (run_id)


8x8Gsn7aI


#### Analysis from the above code

- Random run_id is generated randomly with the help of alphabet function defined above. 
- Server_path variable is initialized with present directory in which the server is running. 
- A folder is created with the name of run_id to store all the files generated with this run time.

__A meta data variable is created to store details of the AutomML models generated. This contains the details of run_id,model start time, model execution time etc__

In [53]:
meta_data = set_meta_data(run_id,data_path,target,run_time,regression,name,run_dir,min_mem_size)
print(meta_data)

{'start_time': 1556319758.330925, 'target': 'score', 'data_path': 'anime_filtered.csv', 'run_time': 333, 'run_id': '8x8Gsn7aI', 'regression': True, 'scale': False, 'project': '8x8Gsn7aIAnime500', 'end_time': 1556319758.3309262, 'execution_time': 0.0, 'run_path': '/Users/vaish/8x8Gsn7aI', 'min_mem_size': 5, 'balance_threshold': 0.2}


In [54]:
meta_data['server_path'] = server_path

__All the independent variables of X and their data types are stored in meta_data__

In [55]:
meta_data['X']=X
allV=get_variables_types(df)
meta_data['variables_datatypes']=allV

__Model execution start time is stored in meta_data__

In [56]:
meta_data['start_time'] = time.time()

__automl function is called and initialized with runtime , x, y , df values__

In [57]:
aml1 = automl(500,X,y,df)

AutoML progress: |████████████████████████████████████████████████████████| 100%


__runtime of the metadata is set to 500 sec__

In [58]:
meta_data['run_time'] = 500

__Total excecution time is calculated by subtracting the start time with the end time__

In [59]:
meta_data['end_time'] = time.time()
meta_data['execution_time'] = meta_data['end_time'] - meta_data['start_time']

__leaderboard is generated for this model__

In [60]:
aml1.leaderboard

model_id,mean_residual_deviance,rmse,mse,mae,rmsle
DRF_1_AutoML_20190426_190238,0.257372,0.507318,0.257372,0.232024,0.102192


__The leaderboard is converted to data frame object so that all model_ids are stored in a seperate list to get parameters__


In [61]:
aml1_leaderboard_df=aml1.leaderboard.as_data_frame()
aml1_leaderboard_df

,model_id,mean_residual_deviance,rmse,mse,mae,rmsle
0,DRF_1_AutoML_20190426_190238,0.257372,0.507318,0.257372,0.232024,0.102192


__Total number of models generated is also stored in meta data__

In [62]:
length = len(aml1_leaderboard_df)
length
meta_data["models_generated"] = length

__Leaderboard is stored in csv files for the excecution time of 500__

In [63]:
# save leaderboard
leaderboard_stats=run_id+'_leaderboard_500.csv'
aml1_leaderboard_df.to_csv(leaderboard_stats)

In [64]:
aml1_leaderboard_df=aml1.leaderboard.as_data_frame()
model1_set=aml1_leaderboard_df['model_id']
model1_set

0    DRF_1_AutoML_20190426_190238
Name: model_id, dtype: object

__Total count of the models is obtained__

In [65]:
count = 0;
for i in model1_set:
    count = count+1;

__Each model parameters are stored in json file along with meta_data__

In [66]:
for i in range(0,count):
    mod_best=h2o.get_model(model1_set[i])
    parameters = mod_best.params
    n= str((model1_set[i]))+'__500'
    dict_to_json(parameters,n)

In [67]:
# Update and save meta data 
n=run_id+'_meta_data.json'
dict_to_json(meta_data,n)

In [68]:
meta_data

{'start_time': 1556319758.347438,
 'target': 'score',
 'data_path': 'anime_filtered.csv',
 'run_time': 500,
 'run_id': '8x8Gsn7aI',
 'regression': True,
 'scale': False,
 'project': '8x8Gsn7aIAnime500',
 'end_time': 1556320262.608652,
 'execution_time': 504.26121401786804,
 'run_path': '/Users/vaish/8x8Gsn7aI',
 'min_mem_size': 5,
 'balance_threshold': 0.2,
 'server_path': '/Users/vaish',
 'X': ['anime_id',
  'episodes',
  'scored_by',
  'rank',
  'popularity',
  'members',
  'favorites',
  'type',
  'source',
  'status',
  'airing',
  'aired_string',
  'aired',
  'duration',
  'rating',
  'premiered',
  'broadcast',
  'related',
  'producer',
  'licensor',
  'studio',
  'genre',
  'opening_theme',
  'ending_theme',
  'title',
  'title_english',
  'title_japanese',
  'title_synonyms',
  'image_url',
  'background'],
 'variables_datatypes': {'anime_id': 'int',
  'title': 'string',
  'title_english': 'string',
  'title_japanese': 'string',
  'title_synonyms': 'string',
  'image_url': 'st

__Since all the files for the particular run id are generated, the directory is again changed to the Server_path variable__



In [69]:
os.chdir(server_path)

## Run Time = 700 seconds

In [70]:
run_id=alphabet(9)
if server_path==None:
    server_path=os.path.abspath(os.curdir)
os.chdir(server_path) 
run_dir = os.path.join(server_path,run_id)
os.mkdir(run_dir)
os.chdir(run_dir)    
name = name = run_id+'Anime'+str('700')
# run_id to std out
print (run_id)

y8WwZr1d7


#### Analysis from the above code

- Random run_id is generated randomly with the help of alphabet function defined above. 
- Server_path variable is initialized with present directory in which the server is running. 
- A folder is created with the name of run_id to store all the files generated with this run time.


__A meta data variable is created to store details of the AutomML models generated. This contains the details of run_id,model start time, model execution time etc__

In [71]:
meta_data = set_meta_data(run_id,data_path,target,run_time,regression,name,run_dir,min_mem_size)
print(meta_data)

{'start_time': 1556320266.520271, 'target': 'score', 'data_path': 'anime_filtered.csv', 'run_time': 333, 'run_id': 'y8WwZr1d7', 'regression': True, 'scale': False, 'project': 'y8WwZr1d7Anime700', 'end_time': 1556320266.520273, 'execution_time': 0.0, 'run_path': '/Users/vaish/y8WwZr1d7', 'min_mem_size': 5, 'balance_threshold': 0.2}


In [72]:
meta_data['server_path'] = server_path

__All the independent variables of X and their datatypes are stored in meta_data. 

In [73]:
meta_data['X']=X 
allV=get_variables_types(df)
meta_data['variables_datatypes']=allV

__Model execution start time is stored in meta_data__

In [74]:
meta_data['start_time'] = time.time()

__automl function is called by giving data,runtimeof 700 ,independent variables and target__

In [75]:
aml2 = automl(700,X,y,df)

AutoML progress: |████████████████████████████████████████████████████████| 100%


__run time of 700 sec is set for the mata data__

In [76]:
meta_data['run_time'] = 700

__excecution time of the meta data is calcualted by subtracting the start and the end time__

In [77]:
meta_data['end_time'] = time.time()
meta_data['execution_time'] = meta_data['end_time'] - meta_data['start_time']

In [78]:
aml2.leaderboard

model_id,mean_residual_deviance,rmse,mse,mae,rmsle
DRF_1_AutoML_20190426_190238,0.257372,0.507318,0.257372,0.232024,0.102192
DRF_1_AutoML_20190426_191106,0.260092,0.509992,0.260092,0.236998,0.102971
GLM_grid_1_AutoML_20190426_191106_model_1,0.804409,0.896889,0.804409,0.557699,0.259754
XRT_1_AutoML_20190426_191106,5.21776,2.28424,5.21776,2.11448,0.413154


__The leaderboard is converted to data frame object so that all model_ids are stored in a seperate list to get parameters__

In [79]:
aml2_leaderboard_df=aml2.leaderboard.as_data_frame()
model2_set=aml2_leaderboard_df['model_id']
model2_set

0                 DRF_1_AutoML_20190426_190238
1                 DRF_1_AutoML_20190426_191106
2    GLM_grid_1_AutoML_20190426_191106_model_1
3                 XRT_1_AutoML_20190426_191106
Name: model_id, dtype: object

__More models are generated by converting the leaderboard to a dataframe which will be further be used to generate a csv file__

In [80]:
length = len(aml2_leaderboard_df)
length
meta_data["models_generated"] = length

__Total number of leaderboards generated are saved as the total number of the models generated for a mata data__ 

In [81]:
# save leaderboard
leaderboard_stats=run_id+'_leaderboard_700.csv'
aml2_leaderboard_df.to_csv(leaderboard_stats)

__Leaderboard generated for the 700 seconds is converted to a csv file__

In [82]:
count = 0;
for i in model2_set:
    count = count+1;

__total count of model sets is taken__

In [83]:
for i in range(0,count):
    mod_best=h2o.get_model(model2_set[i])
    parameters = mod_best.params
    n= str((model2_set[i]))+'__700'
    dict_to_json(parameters,n)

__the parameters of the model generated is saved in the json file__ 

In [84]:
# Update and save meta data

n=run_id+'_meta_data.json'
dict_to_json(meta_data,n)

In [85]:
meta_data

{'start_time': 1556320266.5536761,
 'target': 'score',
 'data_path': 'anime_filtered.csv',
 'run_time': 700,
 'run_id': 'y8WwZr1d7',
 'regression': True,
 'scale': False,
 'project': 'y8WwZr1d7Anime700',
 'end_time': 1556320977.074365,
 'execution_time': 710.5206887722015,
 'run_path': '/Users/vaish/y8WwZr1d7',
 'min_mem_size': 5,
 'balance_threshold': 0.2,
 'server_path': '/Users/vaish',
 'X': ['anime_id',
  'episodes',
  'scored_by',
  'rank',
  'popularity',
  'members',
  'favorites',
  'type',
  'source',
  'status',
  'airing',
  'aired_string',
  'aired',
  'duration',
  'rating',
  'premiered',
  'broadcast',
  'related',
  'producer',
  'licensor',
  'studio',
  'genre',
  'opening_theme',
  'ending_theme',
  'title',
  'title_english',
  'title_japanese',
  'title_synonyms',
  'image_url',
  'background'],
 'variables_datatypes': {'anime_id': 'int',
  'title': 'string',
  'title_english': 'string',
  'title_japanese': 'string',
  'title_synonyms': 'string',
  'image_url': 'st

__Since all the files for the particular run id are generated, the directory is again changed to the server path__

In [86]:
os.chdir(server_path)

## Run Time = 1000 seconds

In [87]:
run_id=alphabet(9)
if server_path==None:
    server_path=os.path.abspath(os.curdir)
os.chdir(server_path) 
run_dir = os.path.join(server_path,run_id)
os.mkdir(run_dir)
os.chdir(run_dir)    

name = run_id+'Anime'+str('1000')
# run_id to std out
print (run_id)

OUVXGfhJL


#### Analysis from the above code

- Random run_id is generated randomly with the help of alphabet function defined above. 
- Server_path variable is initialized with present directory in which the server is running. 
- A folder is created with the name of run_id to store all the files generated with this run time.



__A meta data variable is created to store details of the AutomML models. This contains the details of run_id,model start time, model execution time etc__

In [88]:
meta_data = set_meta_data(run_id,data_path,target,run_time,regression,name,run_dir,min_mem_size)
print(meta_data)

{'start_time': 1556320995.546526, 'target': 'score', 'data_path': 'anime_filtered.csv', 'run_time': 333, 'run_id': 'OUVXGfhJL', 'regression': True, 'scale': False, 'project': 'OUVXGfhJLAnime1000', 'end_time': 1556320995.5465379, 'execution_time': 0.0, 'run_path': '/Users/vaish/OUVXGfhJL', 'min_mem_size': 5, 'balance_threshold': 0.2}


__All the independent variables of X and their datatypes, and runtime,startime of the model are stored in meta_data__



In [89]:
meta_data['server_path'] = server_path
meta_data['X']=X  
allV=get_variables_types(df)
meta_data['variables_datatypes']=allV
meta_data['run_time'] = 1000
meta_data['start_time'] = time.time()

__automl function is called by giving data,runtimeof 1000 ,independent variables and target__

In [90]:
aml3 = automl(1000,X,y,df)

AutoML progress: |████████████████████████████████████████████████████████| 100%


__After the AutoML is executed, execution time of the model is calculated and stored in meta data__

In [91]:
meta_data['end_time'] = time.time()
meta_data['execution_time'] = meta_data['end_time'] - meta_data['start_time']

__These are models generated in an order from best to worst__

In [92]:
aml3.leaderboard

model_id,mean_residual_deviance,rmse,mse,mae,rmsle
DRF_1_AutoML_20190426_190238,0.257372,0.507318,0.257372,0.232024,0.102192
DRF_1_AutoML_20190426_191106,0.260092,0.509992,0.260092,0.236998,0.102971
DRF_1_AutoML_20190426_192315,0.262488,0.512335,0.262488,0.238041,0.102805
GLM_grid_1_AutoML_20190426_191106_model_1,0.804409,0.896889,0.804409,0.557699,0.259754
XRT_1_AutoML_20190426_192315,4.67692,2.16262,4.67692,2.04417,0.361872
XRT_1_AutoML_20190426_191106,5.21776,2.28424,5.21776,2.11448,0.413154


__The leaderboard is converted to data frame object so that all model_ids are stored in a seperate list to get parameters__

In [93]:
aml3_leaderboard_df=aml3.leaderboard.as_data_frame()
model3_set=aml3_leaderboard_df['model_id']
model3_set

0                 DRF_1_AutoML_20190426_190238
1                 DRF_1_AutoML_20190426_191106
2                 DRF_1_AutoML_20190426_192315
3    GLM_grid_1_AutoML_20190426_191106_model_1
4                 XRT_1_AutoML_20190426_192315
5                 XRT_1_AutoML_20190426_191106
Name: model_id, dtype: object

__Total number of models generated is also stored in meta data__

In [94]:
length = len(aml3_leaderboard_df)
length
meta_data["models_generated"] = length

__Leaderboard is stored in csv files__

In [95]:
# save leaderboard
leaderboard_stats=run_id+'_leaderboard_1000.csv'
aml3_leaderboard_df.to_csv(leaderboard_stats)

In [96]:
count = 0;
for i in model3_set:
    count = count+1;

__Each model parameters are stored in json file along with meta_data__

In [97]:
for i in range(0,count):
    mod_best=h2o.get_model(model3_set[i])
    parameters = mod_best.params
    n= str((model3_set[i]))+'__1000'
    dict_to_json(parameters,n)

In [98]:
# Update and save meta data
n=run_id+'_meta_data.json'
dict_to_json(meta_data,n)
meta_data

{'start_time': 1556320995.5639098,
 'target': 'score',
 'data_path': 'anime_filtered.csv',
 'run_time': 1000,
 'run_id': 'OUVXGfhJL',
 'regression': True,
 'scale': False,
 'project': 'OUVXGfhJLAnime1000',
 'end_time': 1556322007.154335,
 'execution_time': 1011.5904252529144,
 'run_path': '/Users/vaish/OUVXGfhJL',
 'min_mem_size': 5,
 'balance_threshold': 0.2,
 'server_path': '/Users/vaish',
 'X': ['anime_id',
  'episodes',
  'scored_by',
  'rank',
  'popularity',
  'members',
  'favorites',
  'type',
  'source',
  'status',
  'airing',
  'aired_string',
  'aired',
  'duration',
  'rating',
  'premiered',
  'broadcast',
  'related',
  'producer',
  'licensor',
  'studio',
  'genre',
  'opening_theme',
  'ending_theme',
  'title',
  'title_english',
  'title_japanese',
  'title_synonyms',
  'image_url',
  'background'],
 'variables_datatypes': {'anime_id': 'int',
  'title': 'string',
  'title_english': 'string',
  'title_japanese': 'string',
  'title_synonyms': 'string',
  'image_url': 

__Since all the files for the particular run id are generated, the directory is again changed to the server path__

In [99]:
os.chdir(server_path)

## Run Time = 1300 seconds

In [100]:
run_id=alphabet(9)
if server_path==None:
    server_path=os.path.abspath(os.curdir)
os.chdir(server_path) 
run_dir = os.path.join(server_path,run_id)
os.mkdir(run_dir)
os.chdir(run_dir)    

name = run_id+'Anime'+str('1300')
# run_id to std out
print (run_id)

RAcsyAaQB


#### Analysis from the above code

- Random run_id is generated randomly with the help of alphabet function defined above. 
- Server_path variable is initialized with present directory in which the server is running. 
- A folder is created with the name of run_id to store all the files generated with this run time.


__A meta data variable is created to store details of the AutomML models. This contains the details of run_id,model start time, model execution time etc__

In [101]:
meta_data = set_meta_data(run_id,data_path,target,run_time,regression,name,run_dir,min_mem_size)
print(meta_data)

{'start_time': 1556322038.6148322, 'target': 'score', 'data_path': 'anime_filtered.csv', 'run_time': 333, 'run_id': 'RAcsyAaQB', 'regression': True, 'scale': False, 'project': 'RAcsyAaQBAnime1300', 'end_time': 1556322038.614835, 'execution_time': 0.0, 'run_path': '/Users/vaish/RAcsyAaQB', 'min_mem_size': 5, 'balance_threshold': 0.2}


__All the independent variables of X are stored in meta_data and model start time is taken into account. And also, datatypes of X variables are stored in meta data__

In [102]:
meta_data['server_path'] = server_path
meta_data['X']=X  
allV=get_variables_types(df)
meta_data['variables_datatypes']=allV
meta_data['run_time'] = 1300
meta_data['start_time'] = time.time()

__automl function is called by giving data,runtime of 1300 ,independent variables and target__



In [103]:
aml4 = automl(1300,X,y,df)

AutoML progress: |████████████████████████████████████████████████████████| 100%


__After the AutoML is executed, execution time of the model is calculated and stored in meta data__

In [104]:
meta_data['end_time'] = time.time()
meta_data['execution_time'] = meta_data['end_time'] - meta_data['start_time']

In [105]:
aml4.leaderboard

model_id,mean_residual_deviance,rmse,mse,mae,rmsle
DRF_1_AutoML_20190426_194039,0.254988,0.504963,0.254988,0.233633,0.101579
DRF_1_AutoML_20190426_190238,0.257372,0.507318,0.257372,0.232024,0.102192
DRF_1_AutoML_20190426_191106,0.260092,0.509992,0.260092,0.236998,0.102971
DRF_1_AutoML_20190426_192315,0.262488,0.512335,0.262488,0.238041,0.102805
GLM_grid_1_AutoML_20190426_191106_model_1,0.804409,0.896889,0.804409,0.557699,0.259754
XRT_1_AutoML_20190426_192315,4.67692,2.16262,4.67692,2.04417,0.361872
XRT_1_AutoML_20190426_191106,5.21776,2.28424,5.21776,2.11448,0.413154


__These are models generated in an order from best to worst__

__The leaderboard is converted to data frame object so that all model_ids are stored in a seperate list to get parameters__

In [106]:
aml4_leaderboard_df=aml4.leaderboard.as_data_frame()
model4_set=aml4_leaderboard_df['model_id']
model4_set

0                 DRF_1_AutoML_20190426_194039
1                 DRF_1_AutoML_20190426_190238
2                 DRF_1_AutoML_20190426_191106
3                 DRF_1_AutoML_20190426_192315
4    GLM_grid_1_AutoML_20190426_191106_model_1
5                 XRT_1_AutoML_20190426_192315
6                 XRT_1_AutoML_20190426_191106
Name: model_id, dtype: object

__Total number of models generated is also stored in meta data__

In [107]:
length = len(aml4_leaderboard_df)
length
meta_data["models_generated"] = length

__Leaderboard is stored in csv files__

In [108]:
# save leaderboard
leaderboard_stats=run_id+'_leaderboard_1300.csv'
aml4_leaderboard_df.to_csv(leaderboard_stats)

In [109]:
count = 0;
for i in model4_set:
    count = count+1;

__Each model parameters are stored in json file along with meta_data__

In [110]:
for i in range(0,count):
    mod_best=h2o.get_model(model4_set[i])
    parameters = mod_best.params
    n= str((model4_set[i]))+'__1300'
    dict_to_json(parameters,n)

In [111]:
n=run_id+'_meta_data.json'
dict_to_json(meta_data,n)
meta_data

{'start_time': 1556322038.62908,
 'target': 'score',
 'data_path': 'anime_filtered.csv',
 'run_time': 1300,
 'run_id': 'RAcsyAaQB',
 'regression': True,
 'scale': False,
 'project': 'RAcsyAaQBAnime1300',
 'end_time': 1556323388.020232,
 'execution_time': 1349.3911519050598,
 'run_path': '/Users/vaish/RAcsyAaQB',
 'min_mem_size': 5,
 'balance_threshold': 0.2,
 'server_path': '/Users/vaish',
 'X': ['anime_id',
  'episodes',
  'scored_by',
  'rank',
  'popularity',
  'members',
  'favorites',
  'type',
  'source',
  'status',
  'airing',
  'aired_string',
  'aired',
  'duration',
  'rating',
  'premiered',
  'broadcast',
  'related',
  'producer',
  'licensor',
  'studio',
  'genre',
  'opening_theme',
  'ending_theme',
  'title',
  'title_english',
  'title_japanese',
  'title_synonyms',
  'image_url',
  'background'],
 'variables_datatypes': {'anime_id': 'int',
  'title': 'string',
  'title_english': 'string',
  'title_japanese': 'string',
  'title_synonyms': 'string',
  'image_url': 's

__Since all the files for the particular run id are generated, the directory is again changed to the server path__

In [112]:
os.chdir(server_path)

## Run Time = 1500 seconds 

In [113]:
run_id=alphabet(9)
if server_path==None:
    server_path=os.path.abspath(os.curdir)
os.chdir(server_path) 
run_dir = os.path.join(server_path,run_id)
os.mkdir(run_dir)
os.chdir(run_dir)    

name = run_id+'Anime'+str('1500')
# run_id to std out
print (run_id)

EN8dwaiSI


#### Analysis from the above code

- Random run_id is generated with the help of alphabet function defined above. 
- Server_path variable is initialized with present directory in which the server is running. 
- A folder is created with the name of run_id to store all the files generated with this run time.


In [114]:
meta_data = set_meta_data(run_id,data_path,target,run_time,regression,name,run_dir,min_mem_size)
print(meta_data)

{'start_time': 1556323429.578681, 'target': 'score', 'data_path': 'anime_filtered.csv', 'run_time': 333, 'run_id': 'EN8dwaiSI', 'regression': True, 'scale': False, 'project': 'EN8dwaiSIAnime1500', 'end_time': 1556323429.5786839, 'execution_time': 0.0, 'run_path': '/Users/vaish/EN8dwaiSI', 'min_mem_size': 5, 'balance_threshold': 0.2}


__A meta data variable is created to store basic functionalities of the AutomML model. This contains the details of run_id,model start time, model execution time__



In [115]:
server_path=os.path.abspath(os.curdir)
meta_data['server_path'] = server_path
meta_data['X']=X 
allV=get_variables_types(df)
meta_data['variables_datatypes']=allV
meta_data['run_time'] = 1500
model_start_time = time.time()

__All the independent variables of X are stored in meta_data and model start time is taken into account. And also, datatypes of X variables are stored in meta data__

In [116]:
meta_data['end_time'] = time.time()
meta_data['execution_time'] = meta_data['end_time'] - meta_data['start_time']

__After the AutoML is executed, execution time of the model is calculated and stored in meta data__

In [117]:
aml5.leaderboard

NameError: name 'aml5' is not defined

__These are models generated in an order from best to worst__

__The leaderboard is converted to data frame object so that all model_ids are stored in a seperate list to get parameters__

In [ ]:
aml5_leaderboard_df=aml5.leaderboard.as_data_frame()
model5_set=aml5_leaderboard_df['model_id']
model5_set

__Total number of models generated is also stored in meta data__

In [ ]:
length = len(aml5_leaderboard_df)
length
meta_data["models_generated"] = length

__Leaderboard is stored in csv files__

In [ ]:
# save leaderboard
leaderboard_stats=run_id+'_leaderboard_1500.csv'
aml5_leaderboard_df.to_csv(leaderboard_stats)

__Each model parameters are stored in json file along with meta_data__

In [ ]:
count = 0;
for i in model5_set:
    count = count+1;

In [ ]:
for i in range(0,count):
    mod_best=h2o.get_model(model5_set[i])
    parameters = mod_best.params
    n= str((model5_set[i]))+'__1500'
    dict_to_json(parameters,n)

In [ ]:
# Update and save meta data  
n=run_id+'_meta_data.json'
dict_to_json(meta_data,n)
meta_data

__Since all the files for the particular run id are generated, the directory is again changed to the server path__

In [ ]:
os.chdir(server_path)

# Conclusion

For each runtime, various models are generated as you can see below.

| RUN TIME (sec) | Algorithms generated | Total number of models |
|---|---|---|
| 333  | StackedEnsemble, Xgboost,GLM | Total number of models |
| 500  | Staked Ensemble, Xgboost, DRF,GBM, GLM | Total number of models |
| 700  | Staked Ensemble, Xgboost, DRF,GBM, GLM | Total number of models |
| 1000 | Staked Ensemble, Xgboost, DRF,GBM, GLM | Total number of models |
| 1300 | Staked Ensemble, Xgboost, DRF,GBM, GLM | Total number of models |
| 1500 | Staked Ensemble, Xgboost, DRF,GBM, GLM | Total number of models |

Ensemble Methods contains combination of other algorithmns and does not have any parameters, hence it is not considered. After getting all the model hyperparameters csv files from database team, we have done Grid Search using XGBRegressor for XGBoost, GradientBoostingRegressor for GBM, ExtraTreesRegressor for XRT, RandomForestRegressor for DRF to find best hyperparameter value among the generated ones.

- For XGBoost - 'col_sample_rate': 0.6,
                'col_sample_rate_per_tree': 0.8,
                 'learn_rate': 0.05,
                  'max_depth': 5,
                  'min_rows': 10.0, 
                  'ntrees': 197,
                  'sample_rate': 0.6,
                  'stopping_rounds': 0 are found to important values for the hyperparameters. 
- For GBM - 'max_depth': 5 

and 

- For GLM,DRF and XRT there were no hyperparameters which can run on Grid Search

Later, ranges for each hyperparameters are found for each algorithm. Few hyperparameters had constant value but few differed like below

- For XGBoost -  'seed' : [-8.186597997308668e+18, 8.50284134061186e+18],
              - 'ntrees': [13.0, 276.0],
              - 'max_runtime_secs': [0.0, 53.0],
              - 'stopping_rounds': [0.0, 3.0],
              - 'max_depth': [5.0, 20.0],
              - 'min_rows': [0.01, 20.0],
              - 'col_sam_r_per_tree': [0.7, 1.0]
- For GBM     - 'max_depth': [3.0, 16.0],
              - 'min_rows': [3.0, 16.0],
              - 'seed': [-9e+18, 8.93e+18],
              - 'sample_rate': [0.5, 1.0]
- For GLM.    - 'glm_id': [1.0, 6.0],
              - 'nfolds': [5.0, 5.0],
              - 'seed': [-5.91e+18, 6.33e+18],
              - 'gradient_epsilon', [1e-06, 1e-06],
- For XRT     - 'seed', [-7.04e+18, 6.82e+18],
              - 'ntrees', [14.0, 50.0],
- For DRF.    - 'seed', [-6.25e+18, 8.56e+18]



# Contributions

- EDA was performed on the dataset so that the dataset is clean and do not contain any null values.
- H2O AutoML is used for the Dataset to get best models for each runtime to store hyperparameters in JSON. 
- For each run time, meta data is generated and stored in json file
- Further, Hyperparameters range and important hyperparameters are found using various criteria like grid search.

- By External source- 30%
- By Vaishnavi - 35%
- By Nupur - 35%

# Citations


https://github.com/prabhuSub/Hyperparamter-Samples

https://github.com/nikbearbrown/CSYE_7245/tree/master/H2O

https://dzone.com/articles/exploring-h2oai-automl

http://docs.h2o.ai/h2o/latest-stable/h2o-docs/automl.html



# License

Copyright 2019 Vaishnavi Malireddy,Nupur Garg 

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.